# AI Wine Sommelier RAG

## Wine Review Indexing

https://www.kaggle.com/datasets/christopheiv/winemagdata130k

In [11]:
%pip install -Uq langchain langchain-openai langchain-pinecone langchain-community

Note: you may need to restart the kernel to use updated packages.


In [12]:
from dotenv import load_dotenv
import os

load_dotenv()  # 현재 경로의 .env를 읽어 환경변수로 등록
os.environ['LANGSMITH_TRACING'] = 'true'  # LangSmith 트레이싱(추적) 기능 활성화
os.environ['LANGSMITH_ENDPOINT'] = 'https://api.smith.langchain.com'  # LangSmith API 엔드포인트 설정
os.environ['LANGSMITH_API_KEY'] = os.getenv('langsmith_key')          # .env의 langsmith_key 값을 LANGSMITH_API_KEY 키로 설정
os.environ['LANGSMITH_PROJECT'] = 'skn23-langchain'                   # LangSmith 프로젝트명 설정
os.environ['OPENAI_API_KEY'] = os.getenv("openai_key")                # .env의 openai_key 값을 OPENAI_API_KEY 키로 설정
os.environ['PINECONE_API_KEY'] = os.getenv("pinecone_key")            # .env의 pinecone_key 값을 PINECONE_API_KEY 키로 설정

## Pinecone 테스트

In [13]:
# 데이터로드
from langchain_core.documents import Document

documents = [
    Document(page_content="LangChain은 LLM 기반 애플리케이션을 쉽게 만들 수 있는 프레임워크입니다.", metadata={"source": "https://langchain.com/docs", "author": "alice", "page": 1}),
    Document(page_content="ChromaDB는 오픈소스 벡터 데이터베이스입니다.", metadata={"source": "https://chromadb.org/intro", "license": "MIT", "date": "2024-07-01"}),
    Document(page_content="파이썬으로 AI 서비스를 개발할 수 있습니다.", metadata={"source": "https://pythonai.co.kr", "editor": "kim", "page": 7}),
    Document(page_content="LLM은 자연어 처리를 위한 대형 언어 모델을 의미합니다.", metadata={"source": "https://llmwiki.com/info", "author": "bob", "version": "v1.1"}),
    Document(page_content="RAG는 검색과 생성의 결합 방식을 제공합니다.", metadata={"source": "https://rag-search.io", "reviewer": "lee", "section": "summary"}),
    Document(page_content="벡터 데이터베이스는 임베딩된 데이터를 효율적으로 검색할 수 있습니다.", metadata={"source": "https://vectorbase.net", "author": "jin", "topic": "vector"}),
    Document(page_content="LangChain을 이용하면 다양한 AI 파이프라인을 구축할 수 있습니다.", metadata={"source": "https://langchain.com/blog", "editor": "sarah", "date": "2024-06-30"}),
    Document(page_content="OpenAI의 GPT 모델은 텍스트 생성에 특화되어 있습니다.", metadata={"source": "https://openai.com/gpt", "lang": "ko", "page": 5}),
    Document(page_content="파이썬은 AI 및 데이터 분석 분야에서 널리 사용되는 언어입니다.", metadata={"source": "https://python.org/usecases", "author": "chun", "updated": "2024-05"}),
    Document(page_content="Streamlit은 파이썬으로 대시보드를 쉽게 만들 수 있는 프레임워크입니다.", metadata={"source": "https://streamlit.io/start", "editor": "park", "date": "2024-04-28"}),
    Document(page_content="Dense Retrieval은 임베딩 벡터를 이용한 검색 방식을 의미합니다.", metadata={"source": "https://retrieval.ai/dense", "type": "tech", "page": 3}),
    Document(page_content="Pandas 라이브러리는 데이터 분석에 자주 사용됩니다.", metadata={"source": "https://pandas.pydata.org/about", "maintainer": "koh", "section": "intro"}),
    Document(page_content="메타데이터 필터링은 검색 결과의 품질을 높여줍니다.", metadata={"source": "https://search.com/metadata", "author": "seo", "feature": "filter"}),
    Document(page_content="SelfQueryRetriever는 자연어 쿼리를 임베딩 쿼리로 변환해줍니다.", metadata={"source": "https://selfquery.ai", "editor": "min", "date": "2024-05-12"}),
    Document(page_content="프롬프트 엔지니어링은 LLM의 성능을 극대화하는 방법입니다.", metadata={"source": "https://prompting.dev/guide", "author": "yang", "topic": "prompt"}),
    Document(page_content="HyDE 기법은 하이브리드 검색에 사용됩니다.", metadata={"source": "https://hyde-tech.com", "reviewer": "kang", "version": "2024.1"}),
    Document(page_content="CoT는 복잡한 문제를 단계적으로 해결하는 프롬프트 기법입니다.", metadata={"source": "https://cotprompt.org", "editor": "jung", "date": "2023-12-01"}),
    Document(page_content="문서 임베딩은 텍스트를 고차원 벡터로 변환하는 과정입니다.", metadata={"source": "https://embedding.ai/intro", "section": "embedding", "author": "song"}),
    Document(page_content="CrewAI는 멀티 에이전트 시스템 구현을 돕는 툴입니다.", metadata={"source": "https://crew.ai/docs", "lang": "ko", "page": 9}),
    Document(page_content="Fine-tuning은 사전학습 모델을 특정 도메인에 맞게 재학습시키는 과정입니다.", metadata={"source": "https://finetune.ai/guide", "editor": "jeon", "date": "2024-01-30"})
]

In [14]:
from langchain_openai import OpenAIEmbeddings       # 문서 텍스트를 임베딩 벡터로 변환
from langchain_pinecone import PineconeVectorStore  # Pinecone 인덱스에 임베딩/문서를 저장하는 벡터스토어

embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

# 문서 -> 임베딩 -> Pinecone 업로드
vector_store = PineconeVectorStore.from_documents(
    documents,    # 업로드할 Document(청크) 리스트
    embeddings,   # 임베딩 모델
    index_name = 'pinecone-first'  # Pinecone 인덱스 이름
)

In [15]:
# 질의를 임베딩 한 뒤 가장 유사한 Document 리스트 반환
retrievals = vector_store.similarity_search('벡터 데이터베이스란?')
retrievals

[Document(id='b68b80f8-7edf-475d-8035-c79f05dceea7', metadata={'author': 'jin', 'source': 'https://vectorbase.net', 'topic': 'vector'}, page_content='벡터 데이터베이스는 임베딩된 데이터를 효율적으로 검색할 수 있습니다.'),
 Document(id='5c3d33d5-a957-4fc6-900f-81813a8210cd', metadata={'date': '2024-07-01', 'license': 'MIT', 'source': 'https://chromadb.org/intro'}, page_content='ChromaDB는 오픈소스 벡터 데이터베이스입니다.'),
 Document(id='8b784f8b-d0ba-4bcc-8b0d-8c0e922268ae', metadata={'page': 3.0, 'source': 'https://retrieval.ai/dense', 'type': 'tech'}, page_content='Dense Retrieval은 임베딩 벡터를 이용한 검색 방식을 의미합니다.'),
 Document(id='e81d544a-a7af-4975-8b16-a9488bb7554d', metadata={'author': 'song', 'section': 'embedding', 'source': 'https://embedding.ai/intro'}, page_content='문서 임베딩은 텍스트를 고차원 벡터로 변환하는 과정입니다.')]

In [16]:
retrievals[1].page_content

'ChromaDB는 오픈소스 벡터 데이터베이스입니다.'

In [17]:
# PineconeVectorStore를 Retriever로 변환해 Top-k 검색
retriever = vector_store.as_retriever(
    search_type = 'similarity',
    search_kwargs = {'k': 3}
)

retriever.invoke('벡터 데이터베이스란?')

[Document(id='b68b80f8-7edf-475d-8035-c79f05dceea7', metadata={'author': 'jin', 'source': 'https://vectorbase.net', 'topic': 'vector'}, page_content='벡터 데이터베이스는 임베딩된 데이터를 효율적으로 검색할 수 있습니다.'),
 Document(id='5c3d33d5-a957-4fc6-900f-81813a8210cd', metadata={'date': '2024-07-01', 'license': 'MIT', 'source': 'https://chromadb.org/intro'}, page_content='ChromaDB는 오픈소스 벡터 데이터베이스입니다.'),
 Document(id='8b784f8b-d0ba-4bcc-8b0d-8c0e922268ae', metadata={'page': 3.0, 'source': 'https://retrieval.ai/dense', 'type': 'tech'}, page_content='Dense Retrieval은 임베딩 벡터를 이용한 검색 방식을 의미합니다.')]

In [18]:
from langchain_community.document_loaders import CSVLoader  # CSV의 각 행(row)를 하나의 Document 객체로 변환하는 로더

loader = CSVLoader('winemag-data-130k-v2.csv', encoding='utf-8')  # CSV 경로/인코딩 지정
docs = loader.load()  # CSV를 Document 리스트로 로드(행 단위)
print(len(docs))


129971


In [19]:
docs = docs[:30000]  # 3만개만 추출
print(len(docs))

30000


In [20]:
# CSV 로더 결과 확인
for i, doc in enumerate(docs[:2], 1):  # Document 중 앞 2개 확인(index 1부터 시작)
    print(f"{i}: {type(doc)}")
    print(f"{doc.metadata}")
    print(f"{doc.page_content}")
    print()

1: <class 'langchain_core.documents.base.Document'>
{'source': 'winemag-data-130k-v2.csv', 'row': 0}
: 0
country: Italy
description: Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.
designation: Vulkà Bianco
points: 87
price: 
province: Sicily & Sardinia
region_1: Etna
region_2: 
taster_name: Kerin O’Keefe
taster_twitter_handle: @kerinokeefe
title: Nicosia 2013 Vulkà Bianco  (Etna)
variety: White Blend
winery: Nicosia

2: <class 'langchain_core.documents.base.Document'>
{'source': 'winemag-data-130k-v2.csv', 'row': 1}
: 1
country: Portugal
description: This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's  already drinkable, although it will certainly be better from 2016.
designation: Avidagos
points: 87
price: 15.0
province: Douro
region_1: 
region_2: 
taster

In [21]:
# 벡터스토어 임베딩 변환 및 업로드
vector_store = PineconeVectorStore(
    index_name = 'winemag-data-130k-v2',  # 업로드 대상 인덱스 이름
    embedding = embeddings                # 임베딩 모델
)

batch_size = 300  # 한 번에 업로드할 Document 개수

for i in range(0, len(docs), batch_size):   # 전체 docs를 batch_size 단위로 순회 (0~300, 300~600, ...)
    batch_data = docs[i: i + batch_size]
    vector_store.add_documents(batch_data)  # 배치 Document들을 임베딩 후 Pinecone 업로드
    print(f'index: {i} ~ {i + batch_size}')

index: 0 ~ 300
index: 300 ~ 600
index: 600 ~ 900
index: 900 ~ 1200
index: 1200 ~ 1500
index: 1500 ~ 1800
index: 1800 ~ 2100
index: 2100 ~ 2400
index: 2400 ~ 2700
index: 2700 ~ 3000
index: 3000 ~ 3300
index: 3300 ~ 3600
index: 3600 ~ 3900
index: 3900 ~ 4200
index: 4200 ~ 4500
index: 4500 ~ 4800
index: 4800 ~ 5100
index: 5100 ~ 5400
index: 5400 ~ 5700
index: 5700 ~ 6000
index: 6000 ~ 6300
index: 6300 ~ 6600
index: 6600 ~ 6900
index: 6900 ~ 7200
index: 7200 ~ 7500
index: 7500 ~ 7800
index: 7800 ~ 8100
index: 8100 ~ 8400
index: 8400 ~ 8700
index: 8700 ~ 9000
index: 9000 ~ 9300
index: 9300 ~ 9600
index: 9600 ~ 9900
index: 9900 ~ 10200
index: 10200 ~ 10500
index: 10500 ~ 10800
index: 10800 ~ 11100
index: 11100 ~ 11400
index: 11400 ~ 11700
index: 11700 ~ 12000
index: 12000 ~ 12300
index: 12300 ~ 12600
index: 12600 ~ 12900
index: 12900 ~ 13200
index: 13200 ~ 13500
index: 13500 ~ 13800
index: 13800 ~ 14100
index: 14100 ~ 14400
index: 14400 ~ 14700
index: 14700 ~ 15000
index: 15000 ~ 15300
index

## Retrieval & Generation
1. 텍스트/이미지 입력으로 요리에 설명 chain
2. 요리설명텍스트 벡터db조회 chain
3. 요리설명/리뷰검색을 가지고 와인추천 응답 chain

### 요리설명 chain

In [22]:
# 멀티모달 체인 구성 : 이미지(여러 장) + 텍스트로 요리 풍미를 50자 이내로 묘사
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate  # (채팅 프롬프트 /휴먼 메시지) 템플릿
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda  # 함수를 Runnable로 감싸 Lanchain 실행

# 입력(dict: text, image_urls)을 기반으로 풍미를 묘사하는 체인을 생성해 반환하는 함수
def describe_dish_flavor(query: dict):
    prompt = ChatPromptTemplate.from_messages([
        # 시스템 지침(페르소나/형식/제한)
        ('system', '''
**페르소나 (Persona):**
당신은 식재료의 분자 단위까지 이해하는 '미식의 철학자'이자, 절대미각을 지닌 최고 수준의 푸드 칼럼니스트이다.
당신은 요리를 단순한 음식이 아닌, 식재료와 조리 과학(Culinary Science)이 빚어낸 예술 작품으로 바라본다.
당신의 표현은 식재료의 기원부터 조리 과정에서 일어나는 화학적 변화(마이야르 반응, 캐러멜라이징 등)를 아우르며, 읽는 이가 마치 그 음식을 입안에 넣은 듯한 착각을 불러일으킬 정도로 정교하고 관능적이다.

**역할 (Role):**
당신의 핵심 역할은 요리의 맛, 향, 텍스처(Texture), 그리고 밸런스를 해부학적으로 분석하여 전달하는 것이다.
1.  **다차원적 분석:** 맛을 평면적으로 묘사하지 않고, '첫맛(Attack) - 중간 맛(Mid-palate) - 끝맛(Finish)'의 시퀀스로 나누어 입체적으로 설명한다.
2.  **조리법과 맛의 인과관계:** 왜 이 맛이 나는지, 어떤 조리 테크닉이 식재료의 잠재력을 폭발시켰는지 논리적 근거를 제시한다.
3.  **미식의 가이드:** 식재료 간의 궁합(Pairing)과 풍미를 극대화하는 팁을 제공하여, 사용자의 미식 수준을 한 단계 끌어올린다.

**가이드라인 (Guidelines):**
- **감각의 구체화:** '맛있다', '부드럽다' 같은 추상적 표현을 금지한다. 대신 '혀를 감싸는 벨벳 같은 질감', '비강을 때리는 훈연 향' 등 구체적인 묘사를 사용하라.
- **단계별 서술:** 시각과 후각으로 시작해, 입안에서의 질감 변화, 그리고 목 넘김 후의 여운까지 단계별로 서술하라.

**예시 (Examples):**

* **사용자:** "잘 만든 '트러플 크림 리조또'의 맛을 묘사해 주세요."
    **당신:**
    * **[시각과 후각]** 김이 모락모락 나는 접시 위로 흙내음(Earthy)을 가득 머금은 트러플 향이 가장 먼저 코끝을 강타합니다. 크림소스의 녹진한 유분 향과 섞여 마치 가을 숲속에 와 있는 듯한 묵직한 아로마가 식욕을 자극합니다.
    * **[첫맛과 텍스처]** 한 숟가락 입에 넣으면, 알덴테(Al dente)로 익혀 심지가 살아있는 쌀알이 혀 위에서 경쾌하게 굴러다닙니다. 동시에 파르미지아노 레지아노 치즈가 녹아든 크림소스가 쌀알 사이사이를 끈적하게 메우며 혀를 포근하게 감싸 안습니다.
    * **[풍미의 폭발]** 씹을수록 버섯의 감칠맛(Umami)이 폭발합니다. 버터의 고소함이 베이스를 깔아주는 가운데, 트러플 오일의 강렬한 향이 비강으로 역류하며 미각을 지배합니다.
    * **[여운]** 목을 넘긴 후에도 트러플의 진한 향과 크림의 고소함이 입안에 길게 남아, 무거운 레드 와인 한 모금을 간절하게 부릅니다.

* **사용자:** "양파 수프(French Onion Soup)의 맛의 비결이 무엇인가요?"
    **당신:**
    * **[핵심 분석]** 이 요리의 영혼은 **'인내심이 만든 단맛'**에 있습니다. 양파를 약불에서 장시간 볶아내는 '캐러멜라이징(Caramelization)' 과정이 핵심입니다.
    * **[맛의 레이어]** 양파의 매운 성분이 열을 만나 짙은 갈색의 끈적한 당분으로 변하며, 설탕과는 차원이 다른 깊고 복합적인 단맛을 냅니다. 여기에 쇠고기 육수의 짭조름한 감칠맛이 더해져 '단짠'의 완벽한 균형을 이룹니다.
    * **[식감의 조화]** 흐물흐물하게 녹아내린 양파와 국물을 머금어 축축해진 바게트, 그리고 그 위를 덮은 그뤼에르 치즈의 쫄깃함이 섞이며 입안 가득 풍성한 식감의 축제를 엽니다.

**주의사항**
맛의 대한 묘사만 줄글 형식으로 50자이내로 작성하세요.

'''),
        # 기본 요청(추가 입력은 아래에서 붙임)
        ('human', '사용자가 제공한 이미지의 요리명과 풍미를 잘 묘사해주세요.')
    ])

    temp = []    # 멀티모달 입력 블록을 담을 리스트
    if query.get('image_urls'):  # query의 image_urls 키가 있는 경우
        temp += [{"image_url": image_url} for image_url in query.get('image_urls')]  # 이미지 urel들을 메시지 블록에 추가
    if query.get('text'):
        temp += [{"text": query.get('text')}]
    
    # 위에서 만든 멀티모달 블록(temp)을 human 메시지로 프롬프트에 추가
    prompt += HumanMessagePromptTemplate.from_template(temp)

    llm = init_chat_model('openai:gpt-4.1-mini')
    output_parser = StrOutputParser()

    chain = prompt | llm | output_parser

    return chain

dish_flavor_chain = RunnableLambda(describe_dish_flavor)  # 입력을 받아 체인을 만들어 실행하는 Runnable

response = dish_flavor_chain.invoke({
    'text': '',
    'image_urls': [
        "https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyNTExMjJfNzUg%2FMDAxNzYzNzk1ODkyNTk1.jLU9W_RNaM7aNBbBC2sWhPu7WpMXXpm6Igwpl6VJj-Ig.OiJgCghMFSrvwDD535qRDqd-tamGpg8YDtachSL1D3Ig.PNG%2Fimage.png&type=l340_165",
        "https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyNTA4MjNfMTU4%2FMDAxNzU1OTE3Nzc0MDUx.-UKTAIZdQbZbQpBDE9iFMKEwAuogl014-j6ze916F6Eg.pPB-5souupVIECA-Eoz4HoPRY5aBxRNRG1Ct6cquwyog.JPEG%2FIMG_0851.jpg&type=a340",
        "https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyNTA1MjJfMjA5%2FMDAxNzQ3OTA1NzAxNDc3.YCyxH1J5SZlveKxrr_9AWVBwRm9P34keO-IvfF2neVMg.Qe6RZdUld6H46VJADwsxlasNM9ZN1Csg5XV3LCibZRog.JPEG%2FR0007281.jpg&type=sc960_832"
    ]  # 요리 사진 URL 여러 장
})

print(response)

1. 돈가스: 바삭한 황금빛 튀김옷이 폭신한 돼지고기 속살을 감싸며, 담백함과 고소함이 조화롭게 어우러진다.
2. 닭갈비: 매콤달콤한 양념이 닭고기에 깊게 배어들어, 씹을수록 진한 감칠맛과 고소한 육즙이 터진다.
3. 설렁탕: 뽀얀 국물이 뼈의 진한 감칠맛을 머금고, 부드러운 고기와 쌀밥이 온화한 풍미를 선사한다.


### 리뷰검색 chain

In [23]:
from langchain_openai import OpenAIEmbeddings         # 질의/문서를 임베딩 벡터로 변환
from langchain_pinecone import PineconeVectorStore    # Pinecone 인덱스에 저장된 벡터 검색

# search_wine_review: 요리(풍미) 설명을 받아 Pinecone에서 유사한 와인리뷰를 찾아 반환하는 함수
def search_wine_review(query):
    embeddings = OpenAIEmbeddings(model='text-embedding-3-small')  # 인덱스와 동일 임베딩 모델 사용
    vector_store = PineconeVectorStore(     # Pinecone 인덱스 연결(검색용)
        index_name='winemag-data-130k-v2',  # 검색 대상 인덱스 이름
        embedding=embeddings                # 질의 임베딩에 사용할 모델
    )

    docs = vector_store.similarity_search(query, k=5)  # 질의를 임베딩해 유사 리뷰 Document 5개 검색

    # print('\n\n'.join(doc.page_content for doc in docs))  # 필요 시 검색된 원문 리뷰를 확인(디버깅용)

		# 후속 체인에서 쓰기 좋게 dict로 구성
    return {
        'dish_flavor': query,  # 입력(요리 풍미 설명)
        # 검색된 리뷰 본문을 하나로 합쳐 반환
        'wine_reviews': '\n\n'.join(doc.page_content for doc in docs)
    }

# 두 음식(스테이크/시저샐러드) 풍미 설명 입력
query = '''
첫 번째 이미지는 '허브 그릴 스테이크'입니다. 입안에서 육즙과 허브 오일이 조화를 이루며 풍부하고 깊은 감칠맛이 혀를 감싸고, 구운 토마토의 은은한 산미가 중간 맛에 신선함을 부여합니다.
두 번째 이미지는 '시저 샐러드'입니다. 크리스피한 크루통과 신선한 로메인 상추가 바삭한 질감을 선사하고, 고소한 파마산 치즈와 크리미한 시저 드레싱이 입안 가득 고소함과 산뜻한 여운을 남깁니다.
'''
search_wine_review(query)  # 함수 실행(결과 dict 반환)

{'dish_flavor': "\n첫 번째 이미지는 '허브 그릴 스테이크'입니다. 입안에서 육즙과 허브 오일이 조화를 이루며 풍부하고 깊은 감칠맛이 혀를 감싸고, 구운 토마토의 은은한 산미가 중간 맛에 신선함을 부여합니다.\n두 번째 이미지는 '시저 샐러드'입니다. 크리스피한 크루통과 신선한 로메인 상추가 바삭한 질감을 선사하고, 고소한 파마산 치즈와 크리미한 시저 드레싱이 입안 가득 고소함과 산뜻한 여운을 남깁니다.\n",
 'wine_reviews': ": 4988\ncountry: US\ndescription: Honey-sweet and direct, with citrus jam, apricot essence, crème brûlée and vanilla cream flavors. Fine with white cookies, lemon chiffon pie, pineapple sorbet.\ndesignation: Madeline\npoints: 88\nprice: 35.0\nprovince: California\nregion_1: Napa Valley\nregion_2: Napa\ntaster_name: \ntaster_twitter_handle: \ntitle: Prager 2004 Madeline Riesling (Napa Valley)\nvariety: Riesling\nwinery: Prager\n\n: 4988\ncountry: US\ndescription: Honey-sweet and direct, with citrus jam, apricot essence, crème brûlée and vanilla cream flavors. Fine with white cookies, lemon chiffon pie, pineapple sorbet.\ndesignation: Madeline\npoints: 88\nprice: 35.0\nprovince: California\nregion_1: Napa Valley\nregion_2: Napa\ntaste

In [24]:
# 풍미 텍스트를 받아 Pinecone에서 리뷰를 찾는 Runnable 생성
search_wine_review_chain = RunnableLambda(search_wine_review)

# 1) 이미지→풍미(텍스트)  2) 풍미→리뷰검색  파이프라인 연결
chain = dish_flavor_chain | search_wine_review_chain

response = chain.invoke({  # 멀티모달 입력(이미지 2장)으로 체인 실행
    "text":"",             # 추가 힌트 텍스트(없으면 빈 문자열)
    "image_urls": [        # 요리 이미지 URL 목록(2장)
        "https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyNTEwMTNfMjM4%2FMDAxNzYwMzYzODc4NDk0.me3_kw-eBIiNd75S0W7XdkBiVbUn78pRz5cVlQWV0EEg.VQcftg1p19qRbhxcvaT9Rk80wha9g1VD0f5yVaIe7cUg.PNG%2FImage_fx_%252866%2529.png&type=sc960_832",
        "https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAxODExMjhfMjEy%2FMDAxNTQzNDE1MzEzOTQw.8BT4PzdMbbRctmFFAkLWz3p3G0KywePJxA70TlwTQjcg.8FTmR-SX6Wt7ey27RbV78uzAD8AqedJJKbaAdduuI3Qg.JPEG.story77616%2F20181128_121945.jpg&type=sc960_832"
    ],
})
print(response)  # 최종 출력(dict: dish_flavor, wine_reviews) 확인

{'dish_flavor': "첫 번째 요리명은 '허브 크러스트 스테이크'입니다.  \n풍미는 진한 육즙 속 탄탄한 마이야르 반응의 카라멜라이징 향과 입안 가득 퍼지는 허브의 상쾌함이 조화를 이룹니다.  \n\n두 번째 요리명은 '시저 샐러드'입니다.  \n풍미는 크리미한 드레싱의 부드러운 산미와 신선한 로메인 특유의 아삭함, 바삭한 크루통의 고소함이 어우러집니다.", 'wine_reviews': ": 2211\ncountry: US\ndescription: A strong note of grapefruit skin and juice, from ruby red to yellow, melds with crushed chalk and cider on the nose of this bottling. The citrus zest is prominent on the palate, as is more apple cider and a solid pithy grip, proving quite intriguing texturally.\ndesignation: Mitzi\npoints: 88\nprice: \nprovince: California\nregion_1: Monterey County\nregion_2: Central Coast\ntaster_name: Matt Kettmann\ntaster_twitter_handle: @mattkettmann\ntitle: Myka 2014 Mitzi Chardonnay (Monterey County)\nvariety: Chardonnay\nwinery: Myka\n\n: 2211\ncountry: US\ndescription: A strong note of grapefruit skin and juice, from ruby red to yellow, melds with crushed chalk and cider on the nose of this bottling. The citrus zest is prominent on the pal

In [ ]:
response = chain.invoke({
    "text":"오늘 저녁은 버터와 허브로 구운 가리비 관자 요리를 먹을 예정입니다."
})
print(response)

In [25]:
### 와인추천 chain

In [26]:
# 와인 추천 체인: (요리 풍미 + 검색된 와인 리뷰)를 근거로 페이링 와인 추천해주는 함수
def recommend_wines(query):
    prompt = ChatPromptTemplate.from_messages([  # system/human 프롬프트 구성
        # 에이전트의 페르소나/가이드라인 지침
        ('system', '''
**페르소나 (Persona):**
당신은 와인과 미식의 조화로운 세계를 탐험하는 '마리아주(Mariage)의 설계자'이자 경험 풍부한 소믈리에이다.
당신은 전 세계의 와인 산지와 품종에 대한 백과사전적 지식을 갖추고 있으며, 복잡한 와인 용어를 누구나 이해하기 쉬운 감각적인 언어로 풀어내는 탁월한 능력을 지녔다.
당신의 태도는 언제나 환대하는 마음(Hospitality)으로 가득 차 있어, 와인 초보자부터 애호가까지 모두를 편안하게 이끈다.

**역할 (Role):**
당신의 유일하고도 가장 중요한 역할은 사용자가 준비한 요리에 **'영혼의 단짝'이 될 와인을 추천**하는 것이다.
1.  **미각 분석:** 요리의 주재료, 소스, 조리법(굽기, 찌기 등)을 분석하여 맛의 무게감과 특성을 파악한다.
2.  **정밀한 페어링:** 산도(Acidity), 당도(Sweetness), 타닌(Tannin), 바디감(Body)의 균형을 고려해 와인을 선정한다.
3.  **이유 설명:** 단순히 와인 이름만 던지는 것이 아니라, **"왜 이 와인이 그 음식과 어울리는지"** 미각적, 화학적 근거를 들어 설득력 있게 설명한다.

**가이드라인 (Guidelines):**
- **음식 중심 예시:** 모든 답변은 구체적인 요리에 대한 와인 추천으로 이루어져야 한다.
- **상호보완의 원리:** 와인이 음식의 맛을 어떻게 상승시키는지(증폭), 혹은 음식의 단점을 어떻게 가려주는지(보완) 묘사하라.

**예시 (Examples):**
... (생략) ...
'''),
				# 입력 변수(dish_flavor, wine_reviews) 기반 요청
        ('human', '''
와인페이링 추천에 있어 아래 제시된 요리와 풍미, 와인리뷰만을 기초하여 답변해주세요.

## 요리와 풍미 ##
{dish_flavor}

## 와인리뷰 정보 ##
{wine_reviews}

'''),
    ])

    llm = init_chat_model('openai:gpt-4.1-mini')  # 사용할 LLM 지정
    output_parser = StrOutputParser()             # 출력 → 문자열 변환

    chain = prompt | llm | output_parser          # 프롬프트 → LLM → 파서 체인 구성

    return chain  # RunnableLambda에서 실행할 체인 반환

recommend_wines_chain = RunnableLambda(recommend_wines)  # 입력 dict를 받아 체인을 만들어 실행하는 Runnable
response = recommend_wines_chain.invoke({                # dish_flavor/wine_reviews를 주입해 실행
    'dish_flavor': "\n첫 번째 이미지는 '허브 그릴 스테이크'입니다. ...\n",
    'wine_reviews': ": 108411\ncountry: US\ndescription: ...\nwinery: Punk Dog"
})
print(response)  # 최종 와인 추천 텍스트 출력

안타깝게도 제공하신 정보에 요리의 구체적인 조리법, 풍미, 그리고 와인 리뷰 내용이 빠져 있어서 최적의 페어링을 추천하기 어렵습니다. 

다만, '허브 그릴 스테이크'라는 요리명을 토대로 감각적인 페어링을 제안드릴 수 있습니다.

---

### 미각 분석  
허브 그릴 스테이크는 육즙이 풍부한 소고기를 그릴 방식으로 조리하여 고소하면서도 약간은 훈연된 풍미가 있습니다. 허브(로즈마리, 타임 등)가 고기의 풍미에 청량하고 상쾌한 향을 더해 균형을 맞추며, 육즙과 함께 입안 가득 진한 감칠맛을 선사합니다.

- **주재료:** 소고기 스테이크  
- **조리법:** 그릴 (불향과 약간의 탄맛)  
- **풍미:** 허브 향, 짭조름함, 육즙의 풍부함, 약한 탄맛과 스모키함  

---

### 와인 리뷰 정보  
'108411', 'Punk Dog'와인에 대한 구체적 설명이 제공되진 않았으나, 미국산 와인이라고 하셨으니 캘리포니아, 오레곤, 혹은 워싱턴 같은 주요 산지의 대표적인 품종 가능성이 높습니다.  
미국산 레드 와인의 대표 격인 카베르네 소비뇽, 진판델, 혹은 쉬라즈(시라)일 확률이 큽니다.

---

### 와인 추천과 이유  

**추천 와인: 미국산 카베르네 소비뇽 또는 쉬라즈 (시라)**  

- **이유:**  
  카베르네 소비뇽은 뚜렷한 타닌감과 검은 과일(블랙커런트, 블랙체리)의 농밀한 맛, 그리고 오크 숙성을 통한 바닐라와 토스트 향이 풍성합니다. 이 타닌은 소고기의 단백질과 만나 입안을 깔끔하게 씻어내어 스테이크의 기름진 감칠맛과 절묘한 상쇄 효과를 냅니다. 허브의 상큼한 향과도 적당히 맞물려 복합적인 미각 경험을 줍니다.  

  쉬라즈는 좀 더 스파이시하고 후추 향이 강하며, 붉은 베리류의 과즙감이 풍부해 그릴에서 올라온 연기 맛과도 훌륭히 어우러집니다. 타닌과 알코올 바디감이 훌륭하여 허브 그릴 스테이크의 육즙과 짭짤한 맛을 효과적으로 강조합니다.

---

### 정리  
- **스테이크의 허브와 불향 <-> 와인의 오크와 스파

### 통합 Chain

In [27]:
# 입력(text/image_urls) → 요리 풍미(텍스트) 생성
dish_flavor_chain = RunnableLambda(describe_dish_flavor)       
# 풍미 텍스트 → Pinecone 유사 와인리뷰 검색(dict 반환)
search_wine_review_chain = RunnableLambda(search_wine_review)
# (dish_flavor + wine_reviews) → 최종 와인 페어링 추천
recommend_wines_chain = RunnableLambda(recommend_wines)

# 3단계 파이프라인 연결
chain = dish_flavor_chain | search_wine_review_chain | recommend_wines_chain

response = chain.invoke({  # 이미지 2장을 넣어 전체 체인 실행
    "text":"",             # 추가 힌트 텍스트(없으면 빈 문자열)
    "image_urls": [        # 요리 이미지 URL 목록(2장)
        "https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyNTEwMTNfMjM4%2FMDAxNzYwMzYzODc4NDk0.me3_kw-eBIiNd75S0W7XdkBiVbUn78pRz5cVlQWV0EEg.VQcftg1p19qRbhxcvaT9Rk80wha9g1VD0f5yVaIe7cUg.PNG%2FImage_fx_%252866%2529.png&type=sc960_832",
        "https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAxODExMjhfMjEy%2FMDAxNTQzNDE1MzEzOTQw.8BT4PzdMbbRctmFFAkLWz3p3G0KywePJxA70TlwTQjcg.8FTmR-SX6Wt7ey27RbV78uzAD8AqedJJKbaAdduuI3Qg.JPEG.story77616%2F20181128_121945.jpg&type=sc960_832"
    ],
})

print(response)  # 최종 와인 추천 텍스트 출력

두 가지 요리와 주어진 와인리뷰를 바탕으로 최적의 와인 페어링을 설계해보겠습니다.

---

## 첫 번째 요리: 허브와 향신료 마리네이드 스테이크
- **풍미 분석:** 
  - 진한 마이야르 반응으로 인한 깊은 육향과 바삭한 겉면 크러스트가 강렬한 풍미와 단단한 바디감을 형성합니다.
  - 허브에서 오는 상큼한 녹내음과 스파이시한 여운이 맛의 마지막을 우아하게 장식합니다.
- **와인 페어링 팁:**
  - 스테이크의 무게감과 진한 육향을 상쇄하면서도, 허브와 스파이스의 상큼함을 보완할 산도 높고 바디감이 중간 이상인 와인 필요.
  - 강한 타닌은 마이야르 반응으로 생긴 씁쓸함과 겹칠 수 있으니 적당한 타닌 수준 권장.

- **추천 와인:**  
  **Myka 2014 Mitzi Chardonnay (Monterey County, California) / 점수: 88**  
  - 리뷰에 따르면 자몽 껍질과 즙의 강렬한 시트러스 노트가 입 안을 활기차게 만들고, 분쇄된 백악과 사과 사이더의 미네랄 감이 텍스처를 살립니다.
  - 이 샤르도네는 크리미함보다는 상큼하고 깔끔한 산미가 강조되어 허브의 녹내음과 강한 육향을 깔끔하게 감싸줍니다.
  - 바삭한 크러스트의 크리미한 느낌을 부드럽게 매칭해주면서 스테이크의 묵직한 풍미와 균형을 맞춥니다.

---

## 두 번째 요리: 크림 드레싱 시저 샐러드
- **풍미 분석:** 
  - 로메인의 아삭함과 시저 소스의 크리미한 질감, 마일드하지만 풍부한 감칠맛, 파마산 치즈의 짭조름함과 고소한 크루통의 식감 대비가 돋보임.
  - 전체적으로 가볍고 신선하나 크림과 치즈에서 오는 지방감이 무겁지 않게 조화를 이룸.

- **와인 페어링 팁:**
  - 크림과 치즈의 부드러움을 살려주면서도, 신선한 산미가 기름진 감을 깔끔히 정리해주는 화이트 와인 권장.
  - 너무 무거운 바디감은 샐러드의 가벼움과 어울리지 않으므로 산미와 과일향이 뚜렷한 와인 선택 필요.

- **추천 와인:**  
  **Jekel 2014 

In [28]:
# 3단계 체인(풍미→리뷰→추천) 실행
response = chain.invoke({
    "text":"이따 베이컨포테이토 피자 먹을건데, 어울리는 와인 좀...",  # 요리/상황 힌트 텍스트
    # 피자 이미지 URL(요리 인식 보조)
    "image_urls": [
        "https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyMTExMjZfOTAg%2FMDAxNjM3ODc3MTI3MDY4.GrivVlwn1aHP8ydQZIA7mym_2AUzq7UaB8zqoTc1d8Mg.z6fe2yK3Tq9E95fHOpHJBq-c3fRUUTGPemiSY939QBcg.JPEG.kkuljo%2F20200406_210410.jpg&type=sc960_832"
    ],
})
print(response)  # 최종 와인 페어링 추천 텍스트 출력

이따 베이컨포테이토 피자의 풍미는 베이컨의 짭조름한 기름기와 감자·치즈의 부드럽고 크리미한 질감, 은은한 옥수수 단맛이 조화를 이루는 까닭에, 이 기름진 무게감을 균형 잡아줄 적당한 산도와 부드러운 질감의 와인이 필요합니다.

제시된 와인리뷰를 참조하면, 두 가지 주요 스타일이 보입니다.

1. **Beaucanon 1999 Chardonnay (Napa Valley)**  
이 샤르도네는 토스티 오크, 바닐라, 버터스카치의 향으로 풍성하며, 멜론, 사과, 꿀, 향신료, 토스트 맛이 크리미한 마무리와 함께 입안을 감쌉니다. 이 와인은 풍부한 바디감과 크리미한 질감이 피자의 치즈와 감자의 부드러움과 완벽히 조화를 이루며, 토스트된 오크와 바닐라 노트가 베이컨의 훈연 풍미와 고소함을 더욱 돋보이게 만듭니다. 다만 산미가 상대적으로 부드럽기에, 기름진 베이컨 맛을 깔끔히 정리해주기보다는 풍성한 맛의 연장선에서 음식과 더 깊은 결을 이룹니다.

2. **Bello Family Vineyards 2011 Chardonnay (Carneros)**  
이 와인은 버터 토스트, 버터스카치, 카라멜, 바닐라 풍미가 강하며, 말로락틱 발효를 통한 부드럽고 크리미한 마무리가 특징입니다. 피자의 크리미한 치즈와 감자, 옥수수 단맛과 매우 잘 맞아, 전반적으로 부드럽고 기름진 맛을 부드럽게 감싸 안으며 편안한 조화를 이룹니다. 하지만 산도는 올라와 있어 베이컨의 기름진 맛을 산뜻하게 다소 정리해주는 정도는 아닙니다.

---

### 추천

**Beaucanon 1999 Chardonnay (Napa Valley)을 강력 추천합니다.**

- 풍부한 오크 풍미와 부드러운 크리미함이 피자의 치즈와 감자의 질감을 자연스럽게 잇고,  
- 토스트와 바닐라의 고소한 향이 훈연된 베이컨의 풍미를 한층 돋운다,  
- 적절한 산도와 꿀 같은 여운이 옥수수의 은은한 단맛과 부드럽게 어우러져 입안을 상쾌하게 마무리해준다.

베이컨포테이토 피자의 재료들이 가진 기름지고 묵직한 풍미 사이를 유연하게

In [29]:
# 3단계 체인(풍미→리뷰→추천) 실행
response = chain.invoke({
    "text":"고추잡채엔 어떤 와인이 어울릴까?",  # 사용자 질문/요리명 힌트
    # 고추잡채 이미지 URL(요리 인식 보조)
    "image_urls": [
        "https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyNTEwMDNfNDYg%2FMDAxNzU5NDkwODcxMjE4.nDH5zOdTi5HvhLUtgYXtl9ps1jQLWn5MNC1XNrChF8Yg.xADP7ofugONTY3vwbnFeS7cQQmPTBXQfjC87WNsFF3Ig.JPEG%2FIMG_3801.JPG&type=sc960_832"
    ],
})
print(response)  # 최종 와인 페어링 추천 텍스트 출력

고추잡채의 풍미는 감칠맛 진한 간장 소스와 아삭한 고추의 청량함, 돼지고기 육즙의 부드러움, 그리고 깨의 고소한 여운이 균형을 이루고 있습니다. 맛의 무게감은 중간 정도에서 약간 가벼운 편에 가까우며, 소스와 재료가 조화롭게 어우러진 복합적인 맛이 특징입니다. 이 요리에는 깔끔한 산도와 은은한 단맛, 라이트한 바디감을 가진 화이트 와인이나 가벼운 레드 와인이 가장 잘 맞습니다.

주어진 와인 리스트 중에서 고추잡채와 최고의 마리아주가 될 와인을 선정하자면,

### 추천 와인:  
**Bodegas Escudero 2008 Becquer (Rioja, Tempranillo Blend)**

#### 이유 설명:  
- 이 와인은 냄새에서 묵직한 흙내음과 말린 자두향이 느껴지며, 타닌은 가볍고, 크랜베리 같은 산뜻한 산미와 약간의 쓴맛, 허브 향이 난다고 평가받았습니다.
- 고추잡채의 깨 고소함과 살짝 단맛 나는 소스에, 이 와인의 산미는 음식의 감칠맛과 청량함을 증폭시키면서 입안을 개운하게 정리해 줍니다.
- 또한 크랜베리의 가벼운 타닌과 약간의 쓴맛, 허브 향은 돼지고기의 육즙과 고추의 알싸한 맛과 잘 어우러져, 깔끔한 마무리감을 줍니다. 
- 이 와인은 바디감이 과하지 않고 드라이한 편이라 고추잡채의 풍미를 눌러서 부담을 주지 않고, 깨의 고소함과 조화를 이루며 음식과 와인 모두가 돋보이도록 돕습니다.

반면,

- Debonné 2009 Cabernet Franc는 타닌과 산도가 강해 고추잡채의 섬세한 맛을 덮어버릴 우려가 있습니다.
- Pagos de Valcerracin 2015 Ribera del Duero는 산미가 강하고 신맛이 강조되어 고추잡채의 단맛과 감칠맛을 방해할 가능성이 큽니다.

따라서 고추잡채의 맛을 온전히 살리고 음식과 와인이 서로를 빛나게 할 최적의 선택은 바로 **Bodegas Escudero 2008 Becquer, Tempranillo Blend**입니다. 이 와인은 스파이시하면서도 부드러운 음식에 풍부한 레드 베리 산미와 가